In [2]:
from datasets import load_dataset

dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

In [7]:
data = dataset['train'][0:500]

In [51]:
test1 = dataset['train'][500:600]

In [16]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [17]:
model_type="bert"
model_name= "bert-base-cased"

In [18]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [31]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "use_early_stopping": True,
    "early_stopping_metric": "mcc",
    "train_batch_size": 128,
    "eval_batch_size": 64
}

In [32]:

model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda = False
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [70]:
a = data['instruction']
b = data['input']
c = data['output']
train = []

for i in range(500):
    temp = {}
    temp['context'] = a[i]
    q_as = []
    ans = {}
    ans['answers'] = [{'answer_start':1, 'text':c[i]}]
    ans['id'] = i+1
    ans['is_impossible'] : False
    ans['question'] = b[i]
    q_as.append(ans)
    temp['qas'] = q_as
    train.append(temp)

In [71]:
a = test1['instruction']
b = test1['input']
c = test1['output']
test = []

for i in range(100):
    temp = {}
    temp['context'] = a[i]
    q_as = []
    ans = {}
    ans['answers'] = [{'answer_start':1, 'text':c[i]}]
    ans['id'] = 600+i+1
    ans['is_impossible'] : False
    ans['question'] = b[i]
    q_as.append(ans)
    temp['qas'] = q_as
    test.append(temp)

In [72]:
model.train_model(train, eval_data=test)

add example index and unique id: 100%|███████████████████████████████████████████████████████| 500/500 [00:00<?, ?it/s]


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
train